## Data Preperation:
1. Summarize our data:
    -head(), describe(), info(), isnull(), value_counts(), shape, ... <br>
    -plt.hist(), plt.boxplot()<br>
    -document takeaways (nulls, datatypes to change, outliers, ideas for features, etc.)<br>

2. Clean the data:
    -rename columns<br>
    -missing values: drop columns with too many missing values, drop rows with too many missing values, fill with zero where it makes sense, and then make note of any columns you want to impute missing values in (you will need to do that on split data).<br>
    -**outlier**: an observation point that is distant from other observations https://www.theanalysisfactor.com/outliers-to-drop-or-not-to-drop/<br>
        -ignore, drop rows, snap to a selected max/min value, create bins (cut, qcut)<br>
    -data errors: drop the rows/observations with the errors, correct them to what it was intended<br>
    -address text normalization issues: correct and standardize tex (e.g. deck 'C' 'c')<br>
    -creating new variables out of existing variables (e.g. z = x - y)<br>
3. Split the data:
    -split our data into train, validate and test sample dataframes<br>
    -why? overfitting: model is not generalizable. It fits the data you've trained it on "too well". 3 points does not necessarily mean a parabola.<br>
    -**train**: in-sample, explore, impute mean, scale numeric data (max() - min()...), fit our ml algorithms, evaluate our models.<br>
    -**validate**: out-of-sample confirm our top models have not overfit, evaluate our top models on unseen data. Using validate performance results, we pick the top 1 model.<br>
    -**test**: out-of-sample, how we expect our top model to perform in production, on unseen data in the future. ONLY USED ON 1 MODEL.

In [1]:
# imports
import wrangle
import pandas as pd
import numpy as np

In [2]:
# aqurie the telco.csv
telco = wrangle.get_telco_data()
telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [3]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

### Initial Take Aways:
1. Need to remove duplicate columns
2. Need to change total_charages into a float since it is a number
3. Need to create dummy columns for all categorical columns

## Remove columns:
- There are columns that are duplicate in the database: They represent the same thing.
1. payment_type_id = payment_type
2. internet_service_type_id = internet_service_type
3. contract_type_id = contract_type
- Are there any columns that are unecessary?

In [4]:
# double check that the columns are infact duplicates: 
pd.crosstab(telco.payment_type_id, telco.payment_type)

payment_type,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
payment_type_id,,,,
1,0,0,2365,0
2,0,0,0,1612
3,1544,0,0,0
4,0,1522,0,0


In [5]:
pd.crosstab(telco.internet_service_type_id, telco.internet_service_type)

internet_service_type,DSL,Fiber optic,None
internet_service_type_id,,,
1,2421,0,0
2,0,3096,0
3,0,0,1526


In [6]:
pd.crosstab(telco.contract_type_id, telco.contract_type)

contract_type,Month-to-month,One year,Two year
contract_type_id,,,
1,3875,0,0
2,0,1473,0
3,0,0,1695


In [7]:
# drop the columns: 
telco.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace=True)

In [8]:
# check that the columns are droped
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  tota

### Change dtypes:
- Total charges needs to be changed into a float:

In [9]:
# change total_charges into a float:
telco.loc[:,'total_charges'] = (telco.total_charges + '0')
telco.total_charges = telco.total_charges.astype(float)

In [10]:
# make sure that total_charges is now a float:
telco.total_charges.dtype

dtype('float64')

## Create a function to do this: 

In [11]:
def clean_telco_data(df):
    """
    This function will:
    - drop duplicate and unessasary columns
    - encode total_charges into a float
    """
    
    # Drop duplicate columns
    df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace=True)
    
    #change total charges into a float
    df.loc[:,'total_charges'] = (df.total_charges + '0')
    df.total_charges = df.total_charges.astype(float)
    
    return df

In [12]:
clean_telco_data(wrangle.get_telco_data())

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.60,593.30,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.90,542.40,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.90,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.00,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.90,267.40,Yes,Month-to-month,Fiber optic,Mailed check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,No,No,13,Yes,No,Yes,No,No,Yes,No,No,No,55.15,742.90,No,One year,DSL,Mailed check
7039,Male,0,Yes,No,22,Yes,Yes,No,No,No,No,No,Yes,Yes,85.10,1873.70,Yes,Month-to-month,Fiber optic,Electronic check
7040,Male,0,No,No,2,Yes,No,No,Yes,No,No,No,No,Yes,50.30,92.75,No,Month-to-month,DSL,Mailed check
7041,Male,0,Yes,Yes,67,Yes,No,Yes,No,Yes,Yes,No,Yes,No,67.85,4627.65,No,Two year,DSL,Mailed check


## Create a functoin to split the data:

In [13]:
# make sure the code works: 
from sklearn.model_selection import train_test_split

In [14]:
# there are different ways to do train, test, split
train_val, test = train_test_split( telco, train_size=0.8, stratify = telco.churn, random_state=1234)
train, val = train_test_split(train_val, train_size=0.7, stratify=train_val.churn, random_state=1234)
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
84,Female,0,No,No,12,Yes,Yes,No,Yes,No,No,Yes,No,Yes,89.75,1052.40,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
5021,Female,0,No,No,70,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,111.95,7795.95,No,Two year,Fiber optic,Credit card (automatic)
1577,Female,0,Yes,No,65,Yes,No,Yes,Yes,Yes,No,No,Yes,No,71.00,4386.20,No,Two year,DSL,Bank transfer (automatic)
6366,Female,0,No,No,6,Yes,Yes,No,No,No,No,Yes,Yes,Yes,93.55,536.40,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
3387,Female,0,No,No,30,Yes,Yes,No,Yes,No,No,Yes,Yes,Yes,100.20,2983.80,Yes,Month-to-month,Fiber optic,Electronic check


In [15]:
def split_telco_data(df):
    '''
    This function will:
    - run clean_telco_data 
    - split the data into train, validate, test
    '''
    df = clean_telco_data(df)
    train_validate, test = train_test_split(df, test_size=0.2, random_state=1234, stratify=df.churn)
    train, validate = train_test_split(train_validate, train_size=0.7, random_state=1234, stratify=train_validate.churn)
    return train, validate, test
    

In [16]:
train, validate, test = split_telco_data(wrangle.get_telco_data())
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
84,Female,0,No,No,12,Yes,Yes,No,Yes,No,No,Yes,No,Yes,89.75,1052.40,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
5021,Female,0,No,No,70,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,111.95,7795.95,No,Two year,Fiber optic,Credit card (automatic)
1577,Female,0,Yes,No,65,Yes,No,Yes,Yes,Yes,No,No,Yes,No,71.00,4386.20,No,Two year,DSL,Bank transfer (automatic)
6366,Female,0,No,No,6,Yes,Yes,No,No,No,No,Yes,Yes,Yes,93.55,536.40,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
3387,Female,0,No,No,30,Yes,Yes,No,Yes,No,No,Yes,Yes,Yes,100.20,2983.80,Yes,Month-to-month,Fiber optic,Electronic check


In [17]:
# to figure out which columns are categorical or numerical:
cat_cols, num_cols = [],[]
for col in telco.columns:
    if telco[col].dtype == 'O':
        cat_cols.append(col)
    else:
        if telco[col].nunique() < 4:
            cat_cols.append(col)
        else:
            num_cols.append(col)

In [18]:
# categorical columns:
cat_cols

['gender',
 'senior_citizen',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

> given this list of catgorical coloums I can now either use map, get_dummies, or replace